# Metocean use case - extract data from dfsu and netcdf files

2D model results stored in NetCDF or Grib can be loaded to ModelSkill using xarray. In this way, MIKE 21 modelresults in dfsu format can easily be compared to model results from third party providers often stored in NetCDF. 

In [ ]:
import xarray as xr
import modelskill as ms

## Observations

2 point observations and 1 track observation from Satellite altimetry. 

In [ ]:
o1 = ms.PointObservation('examples/metocean/HKNA_Hm0.dfs0', item=0, x=4.2420, y=52.6887, name="HKNA")
o2 = ms.PointObservation("examples/metocean/eur_Hm0.dfs0", item=0, x=3.2760, y=51.9990, name="EPL")
o3 = ms.TrackObservation("examples/metocean/Alti_c2_Dutch.dfs0", item=3, name="c2")

## MIKE ModelResult

In [ ]:
mrMIKE = ms.model_result('examples/metocean/HKZN_local_2017_DutchCoast.dfsu', name='MIKE21SW', item=0)
mrMIKE2 = ms.model_result('examples/metocean/DutchCoast_2017_subset_v2.dfsu', name='MIKE21SW_v2', item=0)

## Spatial overview

In [ ]:
ms.plotting.spatial_overview(obs=[o1,o2,o3], mod=mrMIKE);

## NetCDF ModelResult

In [ ]:
fn = "examples/metocean/ERA5_DutchCoast.nc"
#xr.open_dataset(fn)

In [ ]:
mrERA5 = ms.model_result(fn, item="swh", name='ERA5')
mrERA5

In [ ]:
mrERA5.data  # mr contains the xr.Dataset

## Test extract from XArray

1) Extract point 
2) Extract track

In [ ]:
mrERA5.extract(o1)

In [ ]:
mrERA5.extract(o1).plot()

In [ ]:
mrERA5.extract(o3).to_dataframe().head()

## Multi-file ModelResult

Use mfdataset to load multiple files as a single ModelResult. Here from CMEMS Meteo-France.

In [ ]:
fn = "examples/metocean/CMEMS_DutchCoast_*.nc"   # Note the wildcard!
mrCMEMS = ms.model_result(fn, item="VHM0", name='CMEMS')
mrCMEMS

In [ ]:
mrCMEMS.data

## Match multiple models and observations and extract

In [ ]:
ms.plotting.temporal_coverage(obs=[o1,o2,o3], mod=[mrERA5, mrCMEMS, mrMIKE, mrMIKE2]);

In [ ]:
cc = ms.match(obs=[o1,o2,o3], mod=[mrERA5, mrCMEMS, mrMIKE, mrMIKE2])
cc

## Analysis and plotting
Which model is better? 

In [ ]:
sk = cc.skill()
sk.swaplevel().sort_index(level="observation").style()

In [ ]:
sk["urmse"].plot.bar(figsize=(6,3));

In [ ]:
cc.mean_skill().style()

In [ ]:
cc.plot.taylor(figsize=6);